In [1]:
import os
import cv2
import torch
import random
import json
import numpy as np
import torch.nn as nn
from pathlib import Path
from tqdm import tqdm
import albumentations as A
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torchvision.models.video as video_models
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold

/Users/jenishkothari/Northeastern/CS6140/Automatic Highlight Generation/myenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CONFIG = {
    'data_root': '../data/9-classes',
    'frame_height': 224,
    'frame_width': 224,
    'num_frames': 32,

    'batch_size': 4,
    'epochs': 1,
    'learning_rate': 1e-4,
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'fold' : 2,
    'optimizer': 'adam',
    'scheduler': 'cosine',
    'dropout': 0.5,

    'model_type': 'r3d',
    'pretrained': True,
}

# Set seed
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)


In [3]:
def load_video_paths(data_root):
    video_paths = []
    labels = []
    class_names = sorted(os.listdir(data_root))
    label_to_idx = {name: i for i, name in enumerate(class_names)}

    for class_name in class_names:
        class_dir = Path(data_root) / class_name
        for video_file in class_dir.glob("*.mp4"):
            video_paths.append(str(video_file))
            labels.append(label_to_idx[class_name])
    return video_paths, labels, label_to_idx

video_paths, labels, label_to_idx = load_video_paths(CONFIG['data_root'])
idx_to_label = {v: k for k, v in label_to_idx.items()}

In [4]:
class SoccerDataset(Dataset):
    def __init__(self, video_paths, labels, config):
        self.video_paths = video_paths
        self.labels = labels
        self.config = config
        self.transform = A.Compose([
            A.Resize(height=config['frame_height'], width=config['frame_width']),
            A.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989])
        ])

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        path = self.video_paths[idx]
        label = self.labels[idx]
        frames = self._load_video(path)
        return frames, label

    def _load_video(self, path):
        cap = cv2.VideoCapture(path)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        indices = np.linspace(0, frame_count - 1, self.config['num_frames'], dtype=int)
        frames = []

        for i in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if not ret:
                continue
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = self.transform(image=frame)['image']
            frames.append(frame)

        cap.release()

        frames = np.array(frames)
        frames = np.transpose(frames, (3, 0, 1, 2))
        return torch.from_numpy(frames).float()


In [5]:
class R3DClassifier(nn.Module):
    def __init__(self, num_classes, config):
        super(R3DClassifier, self).__init__()

        model_type = config['model_type'].lower()
        pretrained = config['pretrained']
        dropout = config['dropout']

        if model_type == 'r3d':
            self.model = video_models.r3d_18(pretrained=pretrained)
        elif model_type == 'mc3':
            self.model = video_models.mc3_18(pretrained=pretrained)
        elif model_type == 'r2plus1d':
            self.model = video_models.r2plus1d_18(pretrained=pretrained)
        else:
            raise ValueError(f"Unsupported model_type '{model_type}' in CONFIG")

        in_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        return self.model(x)


In [6]:
k_folds = CONFIG['fold']
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

all_fold_accuracies = []

for fold, (train_idx, val_idx) in enumerate(skf.split(video_paths, labels)):
    print(f"\n{'='*30}\n▶️ Fold {fold+1}/{k_folds}\n{'='*30}")

    train_paths_fold = [video_paths[i] for i in train_idx]
    val_paths_fold = [video_paths[i] for i in val_idx]
    train_labels_fold = [labels[i] for i in train_idx]
    val_labels_fold = [labels[i] for i in val_idx]

    train_loader = DataLoader(SoccerDataset(train_paths_fold, train_labels_fold, CONFIG), batch_size=CONFIG['batch_size'], shuffle=True)
    val_loader = DataLoader(SoccerDataset(val_paths_fold, val_labels_fold, CONFIG), batch_size=CONFIG['batch_size'], shuffle=False)

    model = R3DClassifier(num_classes=len(label_to_idx), config=CONFIG).to(CONFIG['device'])

    criterion = nn.CrossEntropyLoss()

    if CONFIG['optimizer'] == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG['learning_rate'])
    elif CONFIG['optimizer'] == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr=CONFIG['learning_rate'], momentum=0.9)
    else:
        raise ValueError("Unsupported optimizer in CONFIG.")    
    
    best_val_acc = 0.0

    for epoch in range(CONFIG['epochs']):
        model.train()
        train_preds, train_targets = [], []

        for videos, labels_batch in tqdm(train_loader, desc=f"Fold {fold+1} - Epoch {epoch+1}"):
            videos, labels_batch = videos.to(CONFIG['device']), labels_batch.to(CONFIG['device'])
            outputs = model(videos)
            loss = criterion(outputs, labels_batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_preds.extend(torch.argmax(outputs, 1).cpu().numpy())
            train_targets.extend(labels_batch.cpu().numpy())

        train_acc = accuracy_score(train_targets, train_preds)

        model.eval()
        val_preds, val_targets = [], []

        with torch.no_grad():
            for videos, labels_batch in val_loader:
                videos, labels_batch = videos.to(CONFIG['device']), labels_batch.to(CONFIG['device'])
                outputs = model(videos)
                val_preds.extend(torch.argmax(outputs, 1).cpu().numpy())
                val_targets.extend(labels_batch.cpu().numpy())

        val_acc = accuracy_score(val_targets, val_preds)
        print(f"✅ Epoch {epoch+1}: Train Acc = {train_acc:.4f}, Val Acc = {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            model_path = f"../models/best_model_fold{fold+1}.pth"
            torch.save(model.state_dict(), model_path)
            print(f"💾 Saved best model for Fold {fold+1}: {model_path}")

    all_fold_accuracies.append(best_val_acc)
    
run_summary = {
            'config': {
                'learning_rate': CONFIG['learning_rate'],
                'batch_size': CONFIG['batch_size'],
                'dropout': CONFIG['dropout'],
                'num_frames': CONFIG['num_frames'],
                'optimizer': CONFIG['optimizer'],
                'k_folds': CONFIG['k_folds']
            },
            'fold_accuracies': all_fold_accuracies,
            'avg_accuracy': np.mean(all_fold_accuracies)
        }

with open(f"run_result_lr{CONFIG['learning_rate']}.json", "w") as f:
    json.dump(run_summary, f, indent=2)



▶️ Fold 1/2


/Users/jenishkothari/Northeastern/CS6140/Automatic Highlight Generation/myenv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/jenishkothari/Northeastern/CS6140/Automatic Highlight Generation/myenv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Fold 1 - Epoch 1:   0%|          | 0/89 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.bar([f"Fold {i+1}" for i in range(k_folds)], all_fold_accuracies, color='skyblue')
plt.axhline(np.mean(all_fold_accuracies), color='orange', linestyle='--', label='Average')
plt.ylim(0, 1)
plt.title("Fold-wise Validation Accuracy")
plt.xlabel("Fold")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()
